In [ ]:
#| default_exp learner

# Learner and Callbacks

> Defines custom fast.ai Callbacks for Indic-CLIP training, specifically for calculating retrieval metrics during validation.

## Colab Setup

In [ ]:
#| hide
# Mount Google Drive (Optional, but recommended for persistent storage)
from pathlib import Path
import sys
import os

try:
    from google.colab import drive
    drive.mount('/content/drive')
    print("Google Drive mounted successfully.")
    # Define PROJECT_ROOT for Colab
    PROJECT_ROOT = Path('/content/drive/MyDrive/Indic-Clip') # Adjust path if needed
    if not PROJECT_ROOT.exists():
        print(f"Warning: Project directory not found at {PROJECT_ROOT}. Please ensure it exists.")
    else:
        # Add project root to sys.path
        if str(PROJECT_ROOT) not in sys.path:
            sys.path.insert(0, str(PROJECT_ROOT))
            print(f"Added {PROJECT_ROOT} to sys.path")
        # Change current working directory
        os.chdir(PROJECT_ROOT)
        print(f"Changed working directory to: {os.getcwd()}")

except ModuleNotFoundError:
    print("Not running in Colab, skipping Drive mount.")
    # Define PROJECT_ROOT for local execution (adjust if needed)
    PROJECT_ROOT = Path.cwd()
    if PROJECT_ROOT.name == 'nbs': PROJECT_ROOT = PROJECT_ROOT.parent
    print(f"Running locally. Project root assumed: {PROJECT_ROOT}")
    if str(PROJECT_ROOT) not in sys.path:
         sys.path.insert(0, str(PROJECT_ROOT))
         print(f"Added {PROJECT_ROOT} to sys.path")
except Exception as e:
    print(f"An error occurred during Colab setup: {e}")
    PROJECT_ROOT = Path('.').resolve()
    print(f"Defaulting project root to current dir: {PROJECT_ROOT}")

Mounted at /content/drive
Google Drive mounted successfully.
Added /content/drive/MyDrive/Indic-Clip to sys.path
Changed working directory to: /content/drive/MyDrive/Indic-Clip


In [ ]:
#| export
# Use try-except for robustness, especially during development/export
try:
    import indic_clip.core
    print("Imported indic_clip.core")
except ModuleNotFoundError:
    print("indic_clip.core not found initially.")
    # Attempt to set sys.path if running in Colab and project cloned
    import sys
    if 'google.colab' in sys.modules:
        project_parent = '/content' # Assuming cloned into /content/indic-clip
        if Path('/content/drive/MyDrive/Indic-Clip').exists():
             project_parent = '/content/drive/MyDrive/Indic-Clip'
        if project_parent not in sys.path:
             sys.path.insert(0, project_parent)
             print(f"Added {project_parent} to sys.path")
        try:
            import indic_clip.core
            print("Imported indic_clip.core after path adjustment.")
        except ModuleNotFoundError:
            print("ERROR: Still cannot find indic_clip.core. Ensure project structure is correct.")
            print("Expected: /content/Indic-Clip/indic_clip/core.py or similar in Drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Google Drive detected, setting PROJECT_ROOT to /content/drive/MyDrive/Indic-Clip
Ensure your project files are located there.
Imported indic_clip.core


In [ ]:
#| hide
# !pip install -qr requirements.txt
# !pip install scikit-learn # Make sure sklearn is installed for metrics

In [ ]:
#| hide
# from nbdev.showdoc import *

In [ ]:
#| export
import torch
import torch.distributed as dist
import numpy as np
import logging

from fastai.callback.core import Callback
from fastai.learner import Learner
from fastai.data.load import DataLoader
from fastcore.all import *
from fastai.torch_core import * # For tensor()
from fastai.text.all import *

try:
    from indic_clip.core import get_logger
    from indic_clip.evaluation.metrics import calculate_retrieval_metrics, calculate_zeroshot_accuracy # Import actual functions
    from indic_clip.loss import ContrastiveLoss
except ModuleNotFoundError:
    print("Could not import project modules in 09_learner.ipynb.")
    logging.basicConfig(level=logging.INFO)
    logger = logging.getLogger(__name__)
    def get_logger(name): return logging.getLogger(name)
    # Define stubs if import fails
    def calculate_retrieval_metrics(*args, **kwargs): return {'mean_recall': 0.0, 'i2t_r@1': 0.0, 't2i_r@1': 0.0}
    def calculate_zeroshot_accuracy(*args, **kwargs): return 0.0

logger = get_logger(__name__)

Reloaded indic_clip.core
Reloaded indic_clip.core
Reloaded indic_clip.core
Reloaded indic_clip.core
Reloaded indic_clip.core


## Retrieval Metric Callback

In [ ]:
#| export
# Removed the stub function as we should import the actual one from evaluation.metrics
# If 11_evaluation_metrics.ipynb is not yet implemented, the try-except block above will define a basic stub.

In [ ]:
#| export
class ContrastiveCallback(Callback):
    """
    Handles fetching model outputs, calculating contrastive loss,
    collecting features for RetrievalMetricCallback, and providing
    compatible preds for default callbacks.
    """
    order = Recorder.order - 1 # Run early after prediction

    def __init__(self):
        self.loss_func = ContrastiveLoss() # Holds the REAL loss function

    def before_validate(self):
        self.learn.img_feats_for_retrieval = []
        self.learn.txt_feats_for_retrieval = []
        logger.debug("ContrastiveCallback: Initialized learner lists for retrieval features.")

    def after_pred(self):
        """
        Intercept raw prediction tuple, calculate local logits for learn.pred,
        store raw tuple for loss, and collect features for retrieval.
        """
        raw_preds = self.learn.pred # Get the original tuple (img, txt, scale)
        self._current_raw_preds = raw_preds # Store for after_loss

        if raw_preds is None:
            logger.warning("ContrastiveCallback.after_pred received None prediction.")
            self.learn.pred = None # Keep it None if input was None
            return

        try:
            # Unpack the raw predictions
            img_feat, txt_feat, logit_scale = raw_preds

            # --- Calculate LOCAL logits to populate learn.pred ---
            # Use features directly from this batch, NOT gathered features here
            local_logits = logit_scale * img_feat @ txt_feat.t()
            self.learn.pred = local_logits # Set learn.pred to something GatherPredsCallback can handle

            # --- Collect features for RetrievalMetricCallback during validation ---
            if not self.training:
                # Ensure lists exist on learner (defensive check)
                if not hasattr(self.learn, 'img_feats_for_retrieval'):
                    self.learn.img_feats_for_retrieval = []
                if not hasattr(self.learn, 'txt_feats_for_retrieval'):
                    self.learn.txt_feats_for_retrieval = []

                self.learn.img_feats_for_retrieval.append(img_feat.cpu().detach())
                self.learn.txt_feats_for_retrieval.append(txt_feat.cpu().detach())

        except Exception as e:
             logger.error(f"Error processing preds in ContrastiveCallback.after_pred: {e}", exc_info=True)
             # Set pred to None or a dummy tensor if unpacking/logit calc fails
             self.learn.pred = None
             # Ensure raw preds are also cleared if they caused error
             if hasattr(self, '_current_raw_preds'): delattr(self, '_current_raw_preds')


    def after_loss(self):
        """
        Calculate the *actual* contrastive loss using stored raw predictions
        (potentially with gathered features) and overwrite learn.loss/loss_grad.
        """
        # Check if we stored preds for this batch
        if hasattr(self, '_current_raw_preds') and self._current_raw_preds is not None:
            try:
                # --- Re-calculate Loss Correctly (using potentially gathered features) ---
                image_features, text_features, logit_scale = self._current_raw_preds

                # gathered_image_features = image_features
                # gathered_text_features = text_features

                # # Calculate similarity and loss (similar to ContrastiveLoss internal logic)
                # logits_per_image = logit_scale * gathered_image_features @ gathered_text_features.t()
                # logits_per_text = logits_per_image.t()

                # global_batch_size = gathered_image_features.size(0)
                # device = image_features.device
                # labels = torch.arange(global_batch_size, device=device, dtype=torch.long)

                # loss_img = F.cross_entropy(logits_per_image, labels)
                # loss_txt = F.cross_entropy(logits_per_text, labels)
                # actual_loss = (loss_img + loss_txt) / 2

                actual_loss = self.loss_func((image_features, text_features, logit_scale))

                # --- End Re-calculation ---

                # Check if loss calculation resulted in NaN
                if torch.isnan(actual_loss):
                    logger.warning(f"NaN detected in calculated loss within ContrastiveCallback (Loss Img: {loss_img.item()}, Loss Txt: {loss_txt.item()}). Setting loss_grad to NaN.")
                    self.learn.loss_grad = actual_loss
                    self.learn.loss = actual_loss.clone().detach()
                else:
                    # Overwrite learner's loss attributes with the valid loss
                    self.learn.loss_grad = actual_loss # For backward pass
                    self.learn.loss = actual_loss.clone().detach() # For recording
                    # logger.debug(f"ContrastiveCallback calculated loss: {self.learn.loss.item()}")

            except Exception as e:
                 logger.error(f"Error calculating loss in ContrastiveCallback.after_loss: {e}", exc_info=True)
                 device = default_device()
                 try: device = self._current_raw_preds[0].device
                 except: pass
                 self.learn.loss_grad = torch.tensor(float('nan'), device=device)
                 self.learn.loss = torch.tensor(float('nan'), device=device)
            finally:
                 # Clean up stored preds for the batch
                 delattr(self, '_current_raw_preds')
        # else: logger.debug("No _current_raw_preds found in after_loss.")

In [ ]:
#| export
class RetrievalMetricCallback(Callback):
    """
    1) Reads features collected by ContrastiveCallback from learner in after_validate,
    2) Computes retrieval metrics,
    3) Stores mean_recall in self._last_metrics,
    4) Injects it into recorder.values in after_epoch.
    """
    order = Recorder.order + 1  # run right after Recorder.after_epoch

    def __init__(self, k_values=[1, 5, 10]):
        store_attr()
        # No longer needs self.img_feats, self.txt_feats
        self._last_metrics = {} # Store metrics calculated in after_validate

    # REMOVED: before_validate (handled by ContrastiveCallback)
    # REMOVED: after_pred (handled by ContrastiveCallback)

    def _gather_features(self, features_list):
        "Gathers features from all GPUs if in distributed mode."
        # Note: This needs testing in a distributed setup.
        # Assumes features_list is a list of tensors on CPU already.
        if dist.is_available() and dist.is_initialized():
            world_size = dist.get_world_size()
            if world_size <= 1: return features_list # No need to gather

            logger.info(f"Gathering features from {world_size} GPUs.")
            gathered_nested = [None] * world_size
            try:
                # Ensure objects are pickleable (CPU tensors should be)
                dist.all_gather_object(gathered_nested, features_list)
                # Concatenate gathered lists
                all_features = []
                for rank_list in gathered_nested:
                    all_features.extend(rank_list)
                logger.info(f"Gathered total {len(all_features)} feature tensors.")
                return all_features
            except Exception as e:
                 logger.error(f"Error during all_gather_object: {e}", exc_info=True)
                 return [] # Return empty list on error
        else:
            # logger.debug("Not in distributed mode or world_size=1. Using local features.")
            return features_list

    def after_validate(self):
        """Concatenate features stored on Learner, compute metrics, stash in self._last_metrics."""
        logger.info("RetrievalMetricCallback: Starting metric calculation in after_validate.")

        # Retrieve features from the Learner attributes
        img_feats_list_local = getattr(self.learn, 'img_feats_for_retrieval', [])
        txt_feats_list_local = getattr(self.learn, 'txt_feats_for_retrieval', [])

        if not img_feats_list_local or not txt_feats_list_local:
            logger.warning("No features found on learner (img_feats_for_retrieval / txt_feats_for_retrieval). Skipping metric calculation.")
            self._last_metrics = {}
            # Clean up learner attributes even if empty
            if hasattr(self.learn, 'img_feats_for_retrieval'): delattr(self.learn, 'img_feats_for_retrieval')
            if hasattr(self.learn, 'txt_feats_for_retrieval'): delattr(self.learn, 'txt_feats_for_retrieval')
            return

        logger.info(f"Found {len(img_feats_list_local)} image feature batches and {len(txt_feats_list_local)} text feature batches on learner.")

        # --- Gather features across GPUs if in distributed mode ---
        # Gather on all ranks, but only calculate on rank 0
        all_img_features_gathered = self._gather_features(img_feats_list_local)
        all_txt_features_gathered = self._gather_features(txt_feats_list_local)

        # --- Proceed with calculation only on rank 0 ---
        is_rank_zero = not (dist.is_available() and dist.is_initialized()) or dist.get_rank() == 0

        if is_rank_zero:
            if not all_img_features_gathered or not all_txt_features_gathered:
                logger.warning(f"Rank 0: No features available after gathering. Skipping metric calculation.")
                self._last_metrics = {}
                # Still need to clean up learner attributes on other ranks if they exist
                # (Cleanup moved outside rank_zero block)
                return

            # --- Concatenate features ---
            try:
                device = next(self.learn.model.parameters()).device
                # Ensure lists contain tensors before concatenating
                if not all(isinstance(t, torch.Tensor) for t in all_img_features_gathered):
                    logger.error("Gathered image features list contains non-Tensor objects.")
                    self._last_metrics = {}; return
                if not all(isinstance(t, torch.Tensor) for t in all_txt_features_gathered):
                    logger.error("Gathered text features list contains non-Tensor objects.")
                    self._last_metrics = {}; return

                img_feats = torch.cat(all_img_features_gathered).to(device)
                txt_feats = torch.cat(all_txt_features_gathered).to(device)
                logger.info(f"Concatenated features. Img shape: {img_feats.shape}, Txt shape: {txt_feats.shape}")

            except Exception as e:
                logger.error(f"Error concatenating or moving features: {e}", exc_info=True)
                self._last_metrics = {}
                # Cleanup moved outside rank_zero block
                return

             # --- Get Logit Scale ---
            if hasattr(self.learn.model, 'logit_scale') and isinstance(self.learn.model.logit_scale, nn.Parameter):
                 # Clamp before exp just in case (already done in model fwd, but belt-and-suspenders)
                 clamped_log_scale = self.learn.model.logit_scale.data.clamp(max=np.log(1 / 0.01)).clone()
                 logit_scale = clamped_log_scale.exp().to(device)
            else:
                 logger.warning("Logit scale not found or not Parameter on model in RetrievalMetricCallback, using default.")
                 logit_scale = tensor(np.exp(np.log(1 / 0.07)), device=device) # Default CLIP init

            # --- Calculate Metrics ---
            try:
                metrics = calculate_retrieval_metrics(
                    image_features=img_feats,
                    text_features=txt_feats,
                    logit_scale=logit_scale,
                    k_values=self.k_values
                )
                logger.info(f"Calculated retrieval metrics: {metrics}")
                self._last_metrics = metrics # Store for after_epoch injection
            except Exception as e:
                 logger.error(f"Error during metric calculation: {e}", exc_info=True)
                 self._last_metrics = {}

        else:
            logger.debug(f"Rank {dist.get_rank()} skipping metric calculation.")
            self._last_metrics = {} # Ensure non-rank-zero doesn't hold stale metrics

        # --- Clean up learner attributes on ALL ranks ---
        # It's important to clean up on all ranks to free memory,
        # especially after gathering which might duplicate lists.
        if hasattr(self.learn, 'img_feats_for_retrieval'):
            delattr(self.learn, 'img_feats_for_retrieval')
            logger.debug(f"Cleaned up learner.img_feats_for_retrieval on rank {dist.get_rank() if dist.is_initialized() else 0}")
        if hasattr(self.learn, 'txt_feats_for_retrieval'):
            delattr(self.learn, 'txt_feats_for_retrieval')
            logger.debug(f"Cleaned up learner.txt_feats_for_retrieval on rank {dist.get_rank() if dist.is_initialized() else 0}")


    def after_epoch(self):
        """Inject mean_recall scalar into recorder.values."""
        # Only rank zero should modify the recorder state significantly
        is_rank_zero = not (dist.is_available() and dist.is_initialized()) or dist.get_rank() == 0

        if is_rank_zero and self._last_metrics: # Only inject if metrics were calculated
            rec = getattr(self.learn, 'recorder', None)
            if rec is None or not hasattr(rec, 'metric_names') or not hasattr(rec, 'values') or not rec.values:
                if getattr(self.learn, 'metrics', None): # Only error if metrics were expected
                   logger.error("Recorder/values missing – cannot inject mean_recall.")
                else:
                   logger.debug("No recorder/metrics - skipping mean_recall injection.")
                self._last_metrics = {} # Clear metrics if they can't be injected
                return

            # Ensure the list for the current epoch exists in recorder.values
            current_epoch_idx = self.learn.epoch # epoch starts at 0
            if len(rec.values) <= current_epoch_idx:
                 logger.warning(f"Recorder.values list (len {len(rec.values)}) too short for current epoch {current_epoch_idx}. Cannot inject metric.")
                 self._last_metrics = {}
                 return

            # Define the metric name we want to inject/update
            metric_name = 'valid_mean_recall'

            # Ensure metric name exists (should be added by Learner init if passed in metrics)
            if metric_name not in rec.metric_names:
                if getattr(self.learn, 'metrics', None):
                    logger.warning(f"Metric '{metric_name}' not in Recorder.metric_names. Appending.")
                    rec.metric_names.append(metric_name)
                else: # No metrics expected, don't inject
                    logger.debug(f"'{metric_name}' not tracked by Recorder. Skipping injection.")
                    self._last_metrics = {}
                    return

            # Find the index for the metric
            try:
                 metric_idx = rec.metric_names.index(metric_name)
            except ValueError:
                 logger.error(f"'{metric_name}' is somehow not in metric_names after check/append. Skipping injection.")
                 self._last_metrics = {}
                 return

            # Calculate the expected index in the *values* list for this epoch
            # Assumes values[-1] order is [train_loss, valid_loss, metric1, metric2, ...]
            # `Recorder.log` usually stores [smooth_loss, valid_loss, metric1_val, ...]
            # `Recorder.final_record` stores [train_loss_final, valid_loss_final, metric1_final,...]
            # Accessing `rec.values[-1]` is the direct list being built.
            # Index should correspond to metric_idx in metric_names
            target_value_idx = metric_idx -1 # metric_names includes 'epoch', values doesn't in the list? No, index should match.

            # Check if the current values list is long enough
            current_values = rec.values[current_epoch_idx] # Values for the *current* epoch
            expected_len = len(rec.metric_names) - 1 # Minus 'epoch'

            # Get the metric value to inject
            metric_val = float(self._last_metrics.get('mean_recall', 0.0))

            try:
                if len(current_values) > target_value_idx:
                     # Value potentially already exists (maybe placeholder if Recorder added it)
                     logger.info(f"Updating recorder.values[{current_epoch_idx}][{target_value_idx}] ({metric_name}) with value {metric_val:.4f}.")
                     current_values[target_value_idx] = metric_val
                elif len(current_values) == target_value_idx:
                     # Append if it's exactly the next expected position
                     logger.info(f"Appending {metric_val:.4f} to recorder.values[{current_epoch_idx}] for {metric_name}.")
                     current_values.append(metric_val)
                else:
                    # The list is shorter than expected, log warning and append
                    logger.warning(f"Recorder values list for epoch {current_epoch_idx} has unexpected length {len(current_values)} (expected at least {target_value_idx}). Appending {metric_name} value defensively.")
                    current_values.append(metric_val)

            except IndexError:
                 logger.error(f"IndexError accessing recorder.values[{current_epoch_idx}][{target_value_idx}]. Appending defensively.")
                 current_values.append(metric_val)
            except Exception as e:
                 logger.error(f"Failed to inject {metric_name}: {e}", exc_info=True)

        # Clear metrics dict after processing on rank 0 or if not rank 0
        self._last_metrics = {}

## Example Usage Notes

In [ ]:
#| eval: false

# How to use this callback with a fastai Learner:

# from fastai.learner import Learner
# from fastai.optimizer import Adam
# from fastai.callback.wandb import WandbCallback
# from fastai.callback.schedule import fit_one_cycle
# from fastai.callback.fp16 import MixedPrecision
# from fastai.callback.progress import ProgressCallback
# from fastai.callback.core import SaveModelCallback

# # Assume:
# # - 'dls' is your DataLoaders object from 04_data_creation
# # - 'model' is your IndicCLIP model instance from 07_model_clip
# # - 'loss_func' is your ContrastiveLoss instance from 08_loss

# # Instantiate the callback
# retrieval_cb = RetrievalMetricCallback(k_values=[1, 5, 10])

# # Add it to your list of callbacks
# cbs = [
#     # Other essential callbacks
#     MixedPrecision(), # If using AMP
#     ProgressCallback(),
#     # WandbCallback(log_preds=False, log_model=True), # Ensure WandB is initialized
#     SaveModelCallback(monitor='valid_mean_recall', comp=np.greater), # Monitor metric logged by RetrievalMetricCallback

#     # Your custom callback
#     retrieval_cb
# ]

# # Create the Learner
# # Make sure to pass the metric function (even a dummy one) to Learner for the name to be registered
# # fastai needs a function reference in metrics list for SaveModelCallback monitor to work correctly
# def valid_mean_recall(logits=None, targets=None): return 0.0 # Dummy metric fn
# learn = Learner(dls, model, loss_func=loss_func, opt_func=Adam, cbs=cbs, metrics=[valid_mean_recall])

# # Start training
# # learn.fit_one_cycle(n_epoch=5, lr_max=1e-4)

# # During validation epochs, RetrievalMetricCallback will compute and log:
# # valid_i2t_r_at_1, valid_t2i_r_at_5, valid_mean_recall etc. via learn.recorder.log
# # These will appear in WandB logs (picked up by WandbCallback's after_epoch) and can be used by SaveModelCallback.

In [ ]:
#| hide
# Run this in terminal if using nbdev
if __name__ == '__main__' and '__file__' not in globals(): # Only run if executing as script
    try:
        import nbdev.export
        nbdev.export.nb_export('09_learner.ipynb', lib_path='./indic_clip', name='learner')
        print("Exported 09_learner.ipynb to indic_clip/learner.py")
    except ImportError:
        print("nbdev not found. Run 'pip install nbdev' and then 'nbdev_export' in terminal.")
    except Exception as e:
        print(f"Error exporting notebook: {e}")